# Combining recency and similarity

<div class="alert alert-block alert-warning">
Replace <code>YOUR_GITHUB_TOKEN</code> in the install script. To get your token follow the instructions in the <a href="../README.md">README.md</a>
</div>

In [ ]:
%pip install  'https://us-central1-data-359211.cloudfunctions.net/github-proxy/superlinked-0.8.0-py3-none-any.whl?token=YOUR_GITHUB_TOKEN'

In addition to being able to use different spaces to embed your data, you can also combine signals from different models to a single index. This allows you to dynamically change the weight of these embeding at query time.

In [1]:
import time

from superlinked.framework.common.schema.schema import schema
from superlinked.framework.common.schema.schema_object import String, Timestamp
from superlinked.framework.dsl.executor.in_memory.in_memory_executor import InMemoryExecutor
from superlinked.framework.dsl.index.index import Index
from superlinked.framework.dsl.query.param import Param
from superlinked.framework.dsl.query.query import Query
from superlinked.framework.dsl.source.in_memory_source import InMemorySource
from superlinked.framework.dsl.space.recency_space import RecencySpace
from superlinked.framework.dsl.space.text_similarity_space import TextSimilaritySpace
from superlinked.framework.storage.entity import Entity

@schema
class Paragraph:
    body: String
    created_at: Timestamp


paragraph = Paragraph()

relevance_space = TextSimilaritySpace(text=paragraph.body, model="all-MiniLM-L6-v2")
recency_space = RecencySpace(timestamp=paragraph.created_at)
paragraph_index = Index([relevance_space, recency_space])

source: InMemorySource = InMemorySource(paragraph)
executor = InMemoryExecutor(sources=[source], indices=[paragraph_index])
app = executor.run()

NOW = int(time.time())
WEEK_AGO = NOW - 60 * 60 * 24 * 7

source.put([{"id": "recent_and_relevant", "body": "Let's talk physics.", "created_at": NOW}])
source.put(
    [
        {
            "id": "recent_but_not_relevant",
            "body": "Bananas.",
            "created_at": NOW,
        }
    ]
)
source.put(
    [{"id": "old_and_relevant", "body": "Let's talk biology.", "created_at": WEEK_AGO}]
)
source.put(
    [{"id": "old_and_not_relevant", "body": "Stone.", "created_at": WEEK_AGO}]
)

query = (
    Query(
        paragraph_index,
        weights={
            relevance_space: Param("relevance_weight"),
            recency_space: Param("recency_weight"),
        },
    )
    .find(paragraph)
    .similar(relevance_space.text, Param("query_text"))
)

Compare the results biased to relevance.

In [2]:
result = app.query(
    query,
    query_text="I am interested in science articles.",
    relevance_weight=1,
    recency_weight=0.1,
)

for entity in result.entities:
    print(entity.id_.object_id)

old_and_relevant
recent_and_relevant
recent_but_not_relevant
old_and_not_relevant


Check results biased to recency.

In [3]:
result = app.query(
    query,
    query_text="I am interested in science articles.",
    relevance_weight=1,
    recency_weight=2,
)

for entity in result.entities:
    print(entity.id_.object_id)

recent_and_relevant
recent_but_not_relevant
old_and_relevant
old_and_not_relevant
